# ROBUST APPROACH: Supervised learning example: regression.

## ☀️ Prediction of photovoltaic generation for self-consumption.

**Objective:** Predict the next day's PV generation of a household, in order to intelligently manage its consumption. 
* We will use historical data of the **target variable** we want to predict (historical PV generation data) and other features that can help to predict the model.



<img src="figures/ml.png" alt="Data center diagram" width="800">

# **0. Import libraries and data**.

In [ ]:
# We import libraries
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')



# We load the input data set
dataset = pd.read_csv('data/regression_PVforecasting.csv', delimiter=';')


# **1. Data analysis: Understanding the data**

It is necessary to visualize and understand the data we are going to work with, as well as to know its characteristics. 

1. How many rows we have? How many attributes are there in the data?  
2. What are these attributes?
3. Is there any missing data?
4. Statistical summary of the input data set.

**How many attributes are there in the data?**

In [ ]:
### Dataset shape
dataset.shape

**What do they mean?**

In [ ]:
# First 5 rows
dataset.head()

In [ ]:
# Last 5 rows
dataset.tail()


**.dtypes** methods is essential for data cleaning and preprocessing

* ``int64`` or ``float64`` → numeric → can be used for math, stats, ML models

* ``object`` → usually text or mixed data → needs preprocessing

* ``datetime64`` → time-aware operations (resampling, trends, etc.)

In [ ]:
# data format
dataset.dtypes

In [ ]:
# Convert localhour in datetime
dataset['localhour'] = pd.to_datetime(dataset['localhour'])

In [ ]:
dataset['localhour']

Let's check the data types again

In [ ]:
# data format
dataset.dtypes

**3. Is any data missing?** A check is made to see if any data is missing, and if so, the count of empty cells in each attribute is performed. In this case, no data is missing.

In [ ]:
# Check for missing data
dataset.isna().sum()

**4. Statistical Summary of the data.**

In [ ]:
dataset.describe()

## Visualize the data.

A visual way to understand the input data. 

1. Boxplots and Density plots
2. Correlation matrix

### Boxplots

The boxplot allows us to identify outliers and compare distributions. In addition, we know how 50% of the values are distributed (inside the box).

In [ ]:
atributos_boxplot = dataset.plot(kind='box', subplots=True, layout=(3, 3), figsize=(15, 10), sharex=False,
                                 sharey=False, fontsize=10)
plt.show()

Add density plots

### **Correlation matrix** 


Why the correlation matrix is useful in Machine learning:

* **Feature selection:** It shows how strongly each input feature is correlated with the target variable.
    * Features with very low correlation (=0) might add little predictive value.
    * Highly correlated features with the target are often more relevant.
* **Detect multicollinearity**: It helps identify features that are strongly correlated with each other.
    * Using highly correlated predictors can make models (especially linear ones) unstable or redundant.
* **Improve interpretability and model performance**. By removing or combining correlated features, you can make the model simpler, faster, and less prone to overfitting.

In [ ]:
### Seaborn visualization library
import seaborn as sns

# Calculation of correlation coefficients
#Pearson for linear correlation
#Spearman for montonous correlation
#https://duchesnay.github.io/pystatsml/auto_gallery/ml_resampling.html
    
corr = dataset.corr(method='pearson') 
# Remove repeated values
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(12, 10))
#Generar Heat Map,
sns.heatmap(corr, annot=True, fmt=".2f")
    # xticks
#plt.xticks(range(len(corr.columns)), corr.columns);
    # yticks
#plt.yticks(range(len(corr.columns)), corr.columns)
    # plot
plt.show()

# 2. Prepare the data.



### Feature selection

We select/add/remove features in an iterative procress of training and testing the model performance


Add ``time`` and ``month`` columns through the datetime column. 
The data is scaled

In [ ]:
# Add month and time columns
dataset['month'] = pd.DatetimeIndex(dataset['localhour']).month
dataset['hour'] = pd.DatetimeIndex(dataset['localhour']).hour
dataset.drop(['localhour'], axis=1, inplace=True)
dataset

Divide the data into **attributes**: X (features) and **tags**: y (target).

In [ ]:
# Features X ; Target y 
X = dataset.drop(['pvgen'], axis=1) 
y = dataset['pvgen']

In [ ]:
X

In [ ]:
y

### Impute missing data

First, let's check if there is missing data we should handle

In [ ]:
print("Missing values in X:")
print(X.isnull().sum())

print("\nMissing values in y:")
print(y.isnull().sum())

# 3. Data separation: Split the data in train-validation-test.

The data are divided into training data ``X_train``, ``y_train``, validation data ``X_val``, ``y_val`` and test data ``X_test``, ``y_test``.


In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2  # percentage of the input data that I will use to validate the model
  
# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    shuffle=False)

In [ ]:
X_train

In [ ]:
y_test

In [ ]:
X_test

#### Where is the missing data?



In [ ]:
print("Missing values in X train, X val and X test:")
print(X_train.isnull().sum())
print(X_val.isnull().sum())
print(X_test.isnull().sum())



### Impute missing data

####  **For specific imputation methods, you can not impute before splitting!**

If you impute before splitting into train/test, you will be using information from the test set (its statistics like mean/median) to fill missing values in the training data. 

That's called **data leakage** — it contaminates your training process and gives over-optimistic results.

* Correct approach when using the entire dataset for data imputation:

    * Split the data → training & testing.
    * Fit the imputer only on the training data.
    * Apply (transform) it to both the training and test sets.
    
    
PS: If you use some other imputation methods, like backfll/interpolation/etc. then there is no data leakage. 

In [ ]:

# Apply mean imputation on training, validation, and test sets
X_train_imputed = X_train.fillna(X_train.mean())
X_val_imputed   = X_val.fillna(X_train.mean())   # use training mean to avoid data leakage
X_test_imputed  = X_test.fillna(X_train.mean())  # same: use training mean

In [ ]:
print("Missing values after mean imputation:\n")

print("X_train missing values per column:")
print(X_train_imputed.isnull().sum(), "\n")

print("X_val missing values per column:")
print(X_val_imputed.isnull().sum(), "\n")

print("X_test missing values per column:")
print(X_test_imputed.isnull().sum())

### Scale the data

Different scalers handle data in different ways, depending on the algorithm and the data’s nature

* **StandardScaler (Z-score normalization)**   ``from sklearn.preprocessing import StandardScaler ``
    * Centers data around mean = 0 and standard deviation = 1.
    * Keeps outliers but rescales the overall distribution.
* **MinMaxScaler (Normalization to a range)**  ``from sklearn.preprocessing import MinMaxScaler``
    * Rescales features to a fixed range (by default [0, 1]).
    * Preserves shape of original distribution, but sensitive to outliers.
* **RobustScaler (less sensitive to outliers)**  ``from sklearn.preprocessing import RobustScaler``
    * Uses median and interquartile range (IQR) instead of mean and std.
    * Great for datasets with outliers or skewed distributions.



For this scenario,  **data is scaled** using the ``MinMaxScaler()`` method, which scales and translates each attribute individually such that it is within the range [0, 1]. This needs to be done when the scales of the attributes are different (e.g. radiation [0, 650], wind speed [2, 15]).

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# scale attributes/features
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_imputed),
    columns=X_train_imputed.columns)
X_train_scaled.head()

#### Apply the scaler to X_val and X_test

In [ ]:
# Transform validation and test data using the same scaler

X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_imputed),
    columns=X_val_imputed.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_imputed),
    columns=X_test_imputed.columns
)

# 4. Model building and evaluation.

* The selected evaluation metrics are **RMSE and R2**.
* Check all the available evaluation metrics in Scikit Learn https://scikit-learn.org/stable/modules/model_evaluation.html
![available evaluation metrics](./Figures/regression_evaluationmetric.png)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# number of folds for cross-validation

num_folds =3  # To make the code run faster in class, I have reduced the number of folds. But this number should be somewhat higher. 

error_metrics = {'r2'}
models = {('MLP', MLPRegressor()),('RFR', RandomForestRegressor()), ('KNN', KNeighborsRegressor())}



Each of the models is trained, the results are saved and compared visually.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV

# Cross-validation training
for scoring in error_metrics:
    results = [] # store metrics results
    msg = []  # print summary of result
    names = []  # store name of the models
    print('Evaluation metric: ', scoring)
    for name, model in models:
        print('Model ', name)
        cross_validation = TimeSeriesSplit(n_splits=num_folds)
        cv_results = cross_val_score(model, X_train_scaled, y_train, cv=cross_validation, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    print(msg)

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

    results = []


In [ ]:
msg

# 5. Best models hyperparameters adjustment.

Steps to perform the hyperadjustment of the parameters:
* Specify the model to be adjusted
* Specify a metric to optimize
* Define the search parameter ranges: *params*
* Assign a validation method: *KFold*
* Find the Hyperparameters with the validation data: *X_val*


**To make the code run faster in class, we have only used two possible values for the ``n_estimators`` hyperparameter. In reality, the more hyperparameters you test, the better. And the more values per hyperparameter, too. But this can be computationally very intensive. Find a good trade-off.**

In [ ]:
modelo = RandomForestRegressor()
scoring='r2'


params = {
    # Number of trees in random forest
    'n_estimators': [100, 200, 500, 1000],  # default=100
     # Maximum number of levels in tree
    'max_depth': [2, None],  #deafult = None
     # Method of selecting samples for training each tree
}


# Search for the best combination of hyperparameters
cross_validation = TimeSeriesSplit(n_splits=5)
my_cv = cross_validation.split(X_val_scaled)

# Grid search with verbose output
gsearch = GridSearchCV(
    estimator=modelo,
    param_grid=params,
    scoring=scoring,
    cv=my_cv,
    verbose=2,    #  enables detailed progress logs
    n_jobs=-1     #  uses all available CPU cores
)


print("Starting grid search...\n")
gsearch.fit(X_val_scaled, y_val)
print("\nGrid search completed.")


# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']

# 6. Final evaluation of the model.


Finally, PV generation predictions are made.

Evaluation metrics:
  * RMSE
  * R2

    
The ``fit()`` model is trained with the optimal hyperparameters found in the previous section and then the predictions are made. 

In [ ]:
final_model = RandomForestRegressor(n_estimators=100, max_depth= None) ## train again with the winner model from the Grid Search
final_model.fit(X_train_scaled,y_train)  # Model training 



### Now the model is trained, congrats! Let's make predictions with that model, and see if the outcomes are good or not

In [ ]:

y_predict = final_model.predict(X_test_scaled)  # prediction calculation


In [ ]:
y_predict

### Evaluation metrics 

In [ ]:
import math 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Error RMSE de test  
print('RMSE: ', math.sqrt(mean_squared_error(y_test, y_predict)))
print('R2: ', r2_score(y_test, y_predict))

R2 Evaluation metrics: 
* Train 0.91
* Val 0.95
* Test 0.88

* --> There is no overfitting

## Graph results obtained. 

In [ ]:
# Plot y_predict vs y_test

x = range(len(y_predict))
plt.figure(figsize=(20,5))
plt.xlabel('Time', size=15)
plt.ylabel('Energy produced (kWh)', size=15)
plt.plot(x, y_predict, alpha=0.4, color='blue', label='PV predict')
plt.plot(x, y_test, alpha=0.4, color='red',  label='PV real')
plt.title('Prediction vs Real')
plt.legend()
plt.show()

### We need to Zoom in!

If necessary, install the Plotly library ``!pip install plotly``.


In [ ]:
import plotly.graph_objects as go  # Importamos la librería de plotly

init = list(range(len(y_predict)))
y_predict_plot = pd.DataFrame(data=y_predict, index=init, columns=['predict'])

# Reindex y_test so it starts from 0, ensuring y_predict and y_test share the same index for plotting
y_test_plot = pd.DataFrame(data=y_test.values, index=init, columns=['test'])


# We create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=init, y=y_predict_plot['predict'][init],
                    mode='lines',
                    name='PV prediction'))
fig.add_trace(go.Scatter(x=init, y=y_test_plot['test'][init],
                     mode='lines', name='PV real'))


# We edit figure
fig.update_layout(autosize=False,
                  width=1000,
                    height=500,
                    title='Prediction vs Real',
                   xaxis_title='Periods',
                   yaxis_title='Energy (kWh)')


fig.show()

### Extra: Most important features/attributes 

Which features carry the most weight in this example? 

* Not to be confused with correlation matrix, which is not the same thing. For more info: tree.feaure_importances https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html 

* This does not work with all ML regression models.



In [ ]:

# We print the feature ranking
importances = gsearch.best_estimator_.feature_importances_
std = np.std([tree.feature_importances_ for tree in gsearch.best_estimator_.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
feat = X.columns
feat_or=[]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]])+feat[indices[f]])
    feat_or.append(feat[indices[f]])

# We plot the weight of the features that matter most for RandomForest()
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), feat_or, rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()